# Objet : Test opendata des données de qualité de l'air

## Objectif

- valider sur des cas réels l'outil de traitement des "listes indexées"
- identifier les apports que pourraient avoir ce type d'outil

## Résultats
- à compléter

## Usages possibles 
- à compléter

## Autres points
- chargement sur MongoDB à tester
- Outil de requète à tester

données utilisées : https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/temps-reel/2022/

------
## Initialisation
- lecture des fichiers de 01/2022 issus de l'api (un fichier par jour)

In [1]:
from pprint import pprint
from time import time
from datetime import datetime
import csv
from observation import Ilist, Iindex, util, Observation, ESSearch
from copy import copy
import pandas as pd
from test_mongo import clientMongo

chemin = 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/air/data_lcsqa/'
collec = clientMongo()['air_quality']['lcsqa_2']

In [ ]:
envoi = False
nb_fichiers = 1
annee = 2022
mois = 1
jour = 1
# chargement des fichiers du mois et envoi dans mongoDB
t0 = time()
for i in range(nb_fichiers):
    file = chemin + 'FR_E2_' + str(annee) + '-' + format(mois, '02d') +'-' + format(jour + i, '02d') +'.csv'
    data = pd.read_csv(file, sep=';')
    data[['Date de début','Date de fin']] = data[['Date de début','Date de fin']].astype('datetime64')
    data = data.astype('category')
    print('data : \n', len(data), '\n', list(data), time()-t0, '\n')
    if envoi:
        dic = data.to_dict(orient='records')
        name = 'air quality {:4d}-{:02d}-{:02d}'.format(annee, mois, jour + i)
        id = hash(time())
        for js in dic:
            js['_metadata'] = {'id':id, 'name': name}
        print('début mongo ', time()-t0)
        collec.insert_many(dic)
        print('mongo ok ', time()-t0)

In [7]:
# définition des critères de recherche
periode_debut = ['Date de début', datetime(2022, 1, 1, 10), '>='   ]
periode_fin   = ['Date de fin',   datetime(2022, 1, 1, 12), '<='   ]
lieu          = ['nom site',      'MARSEILLE',              'regex']
polluant      = ['Polluant',      'PM',                     'regex']

t0 = time()
#result = ESSearch(collec, []).execute('idfused')
#result = ESSearch(collec, [periode_fin]).execute('idfused')
#result = ESSearch(collec, [periode_debut, periode_fin]).execute('idfused')
result = ESSearch(collec, [periode_debut, periode_fin, lieu, polluant]).execute('idfused')[0]
result.delindex(savename=['Date de début', 'nom site', 'Polluant', 'valeur', 'unité de mesure'])

result.view(tablefmt='html')

Date de début,nom site,Polluant,valeur,unité de mesure
2022-01-01 10:00:00,MARSEILLE RABATAU,PM10,26.90,µg-m3
2022-01-01 11:00:00,MARSEILLE RABATAU,PM10,19.60,µg-m3
2022-01-01 10:00:00,MARSEILLE RABATAU,PM2.5,19.40,µg-m3
2022-01-01 11:00:00,MARSEILLE RABATAU,PM2.5,8.00,µg-m3
2022-01-01 10:00:00,MARSEILLE ST LOUIS,PM10,16.40,µg-m3
2022-01-01 11:00:00,MARSEILLE ST LOUIS,PM10,21.00,µg-m3
2022-01-01 10:00:00,MARSEILLE ST LOUIS,PM2.5,15.80,µg-m3
2022-01-01 11:00:00,MARSEILLE ST LOUIS,PM2.5,19.50,µg-m3
2022-01-01 10:00:00,MARSEILLE 5 AVENUES,PM10,18.10,µg-m3
2022-01-01 11:00:00,MARSEILLE 5 AVENUES,PM10,19.20,µg-m3
